In [20]:
import argparse
from bertviz import model_view
import jsonlines
import os
import numpy as np
import pytorch_lightning as pl

from constants import ENTITY_START_MARKER, ENTITY_END_MARKER
from data_loader import  DrugSynergyDataModule, make_fixed_length
from model import RelationExtractor, load_model
from preprocess import create_dataset
from utils import construct_row_id_idx_mapping, set_seed, write_error_analysis_file

from streamlit_single_relation_app import classify_message

In [21]:
checkpoint_path = "/home/vijay/drug-synergy-models/checkpoints_more_data_with_drug_tokens_paragraph_2024"

In [22]:
model, tokenizer, metadata = load_model(checkpoint_path, output_attentions=True)

In [23]:
tokenizer.add_tokens([ENTITY_START_MARKER, ENTITY_END_MARKER])
drugs = open("drugs.txt").read().lower().split()
tokenizer.add_tokens(drugs)

2377

In [56]:
message_text = "Various cutaneous side-effects have been reported with anti-melanoma systemic therapies. This study investigated the changes in melanocytic lesion pigmentation in patients on four different therapies. ### methods We analysed the serial dermatoscopic photographs of atypical melanocytic lesions taken from patients with advanced metastatic melanoma on four different systemic therapies ( selective BRAF-inhibitor monotherapy , <<m>> dabrafenib <</m>> combined with <<m>> trametinib <</m>> [ D&T ] , anti-programmed cell death protein 1 [ anti-PD1 ] therapies , and anti-PD1 combined with ipilimumab ) seen from February 2013 to May 2016 . We compared these changes with the melanocytic lesions of 10 control patients. ### results In the control group, 19% of naevi lightened, 64% did not change and 17% darkened. Only the BRAF inhibitor group showed more darkened lesions than controls (37%, P < 0.001). Meanwhile, there were more lightened naevi in the D&T therapy group (86%, P < 0.001) as well as the anti-PD1 and ipilimumab groups (59%, P < 0.001) than controls. Patients on anti-PD1 monotherapy had more lightened (49%) and fewer darkened naevi (9%) than controls, but differences were not significant. ### conclusions Our study showed that different anti-melanoma systemic therapies have different effects on the pigmentation of melanocytic lesions. BRAF inhibitor may have the propensity to cause darkening while D&T therapy and anti-PD1 caused lightening compared with controls. The findings emphasise the importance of regular dermatological monitoring in specialised clinics for patients on anti-melanoma systemic therapy. Clinicians should expect changes in the global pigmentation of melanocytic lesions but be suspicious of lesions with structural changes."

In [57]:
inputs = tokenizer.encode(message_text, return_tensors='pt')
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 

In [58]:
#tokens = make_fixed_length(tokens, metadata.max_seq_length, padding_value = "<PAD>")

In [59]:
model_output, attention = classify_message(message_text, model, tokenizer, metadata.max_seq_length)

In [60]:
model_output

{'predicted label': 1, 'relation probabilities': [0.3897, 0.6103]}

In [61]:
first_attention_layer = attention[0]

In [62]:
def only_keep_nth_head(attention_layer, head_idxs, seq_length):
    return attention_layer[:, head_idxs, :seq_length, :seq_length]

In [63]:
attention[0].shape

torch.Size([1, 12, 512, 512])

In [64]:
attention_pruned = (only_keep_nth_head(attention[0], list(range(12)), len(tokens)), only_keep_nth_head(attention[1], list(range(12)), len(tokens)))

In [65]:
attention_pruned[0].shape

torch.Size([1, 12, 371, 371])

In [66]:
[i for i, v in enumerate(tokens) if v == "<<m>>"]

[69, 74]

In [67]:
start_token_idxs = [i for i, v in enumerate(tokens) if v == "<<m>>"]

In [69]:
def print_top_attended_words(attention_tensors, query_token_idx, layer_idx, attn_head_idx, num_words_to_show=10):
    top_self_attentions = np.argsort(-attention_tensors[layer_idx][0][attn_head_idx][query_token_idx].detach())[:num_words_to_show]
    for other_token_idx in top_self_attentions:
        word = f"{tokens[other_token_idx]} ({other_token_idx})"
        if len(word) <= 7:
            num_tabs = 4
        elif len(word) >= 16:
            num_tabs = 2
        else:
            num_tabs = 3
        tabs = "".join(["\t" for _ in range(num_tabs)])
        print(f"{tokens[other_token_idx]} ({other_token_idx}){tabs}{attention_pruned[layer_idx][0][attn_head_idx][start][other_token_idx]}")

TRANSFORMER_LAYER_IDX = 0
ATTENTION_HEAD_IDX = 4
for attention_head in range(12):
    start = start_token_idxs[0]
    print(f"Showing attention head {attention_head} in transformer layer {TRANSFORMER_LAYER_IDX}")
    print(f"Query entity: entity marker before \"{tokens[start+1]}\" ({start})")
    print("===================================================")
    print(f"Subword\t\t\t|\tAttention weight\n---------------------------------------------------")
    print_top_attended_words(attention_pruned, start, TRANSFORMER_LAYER_IDX, attention_head)
    print("\n\n")

Showing attention head 0 in transformer layer 0
Query entity: entity marker before "dabrafenib" (69)
Subword			|	Attention weight
---------------------------------------------------
melanoma (344)			0.016837473958730698
melanoma (281)			0.013962700963020325
melanoma (56)			0.012813893146812916
braf (64)			0.010214664973318577
[SEP] (370)			0.009979208000004292
) (249)				0.009928843006491661
melanoma (12)			0.009653931483626366
) (260)				0.009097267873585224
ipilimumab (219)		0.008931697346270084
) (231)				0.008475184440612793



Showing attention head 1 in transformer layer 0
Query entity: entity marker before "dabrafenib" (69)
Subword			|	Attention weight
---------------------------------------------------
different (59)			0.06901772320270538
different (31)			0.04565685987472534
different (278)			0.03442399948835373
different (285)			0.03403519093990326
analysed (39)			0.025331266224384308
<<m>> (74)			0.024301541969180107
<<m>> (69)			0.016506988555192947
taken (50)			0.0132584776

# model_view(attention_pruned, tokens[:metadata.max_seq_length])